In [1]:
%reset

In [ ]:
import sys
from pathlib import Path

# Current working dir  …/DGRB Scripts/Test - sbi for 1 SFG and 1 mAGN…
# parent[0] → …/DGRB Scripts
# parent[1] → …/home/users/ids29           ← where “DGRB/” lives
package_path = Path.cwd().parents[1] / "DGRB"   # /home/users/ids29/DGRB

sys.path.insert(0, str(package_path))           # make it import-able


In [3]:
import aegis
import numpy as np
import healpy as hp
import torch
import pickle as pk
from astropy import units as u
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
import pickle
from scipy.stats import norm
from scipy.integrate import quad, simpson
from joblib import Parallel, delayed

%matplotlib inline

In [4]:
grains=1000
num_simulations = 1000
num_workers = -1

In [5]:
parameter_range = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range = [1000, 100000] #MeV
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*18]
max_radius = 8.5 + 20*2 #kpc
exposure = 2000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut = np.pi #10*u.deg.to('rad') #degrees
angular_cut_gen = np.pi #angular_cut*1.5
lat_cut = 0 #2*u.deg.to('rad') #degrees
lat_cut_gen = lat_cut*0.5

In [6]:
my_cosmology = 'Planck18'
z_range = [0, 14]
luminosity_range = 10.0**np.array([37, 50]) # Minimum value set by considering Andromeda distance using Fermi as benchmark and receiving 0.1 photon at detector side
my_AEGIS = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range, energy_range, luminosity_range, max_radius, exposure, angular_cut, lat_cut, flux_cut, energy_range_gen=energy_range_gen, cosmology = my_cosmology, z_range = z_range, verbose = False)
my_AEGIS.angular_cut_gen, my_AEGIS.lat_cut_gen = angular_cut_gen, lat_cut_gen

In [ ]:
Gamma_SFG = 2.2
gamma_energy_bounds = energy_range_gen  # in MeV
E_photon_MeV_SFG = ((-Gamma_SFG + 1) / (-Gamma_SFG + 2) *
                (gamma_energy_bounds[1]**(-Gamma_SFG + 2) - gamma_energy_bounds[0]**(-Gamma_SFG + 2)) /
                (gamma_energy_bounds[1]**(-Gamma_SFG + 1) - gamma_energy_bounds[0]**(-Gamma_SFG + 1))) # in MeV
E_photon_SFG = E_photon_MeV_SFG * 1.60218e-6  # erg

Gamma_mAGN = 2.2 # enforced by user to match SFG spectrum (actually should 2.25)
gamma_energy_bounds = energy_range_gen  # in MeV
E_photon_MeV_mAGN = ((-Gamma_mAGN + 1) / (-Gamma_mAGN + 2) *
                (gamma_energy_bounds[1]**(-Gamma_mAGN + 2) - gamma_energy_bounds[0]**(-Gamma_mAGN + 2)) /
                (gamma_energy_bounds[1]**(-Gamma_mAGN + 1) - gamma_energy_bounds[0]**(-Gamma_mAGN + 1))) # MeV
E_photon_mAGN = E_photon_MeV_mAGN * 1.60218e-6  # erg

res = int(1e4)
log_LIRs = np.linspace(-5, 25, res)
log_L5Gs = np.linspace(20, 55, res)

In [8]:
def ZL_SFG1(z, l, params):


    log_PhiStar = params[0]
    Phi_star = 10**log_PhiStar

    l_erg = l * E_photon_SFG # erg/s
    LFs = np.zeros_like(l)

    def Phi_IR(log_LIR): #log_LIR = log_10(L_IR / solar_luminosity) # unitless

        # from Table 8 in Gruppioni et al.
        # Phi_star = 10**(-2.08) # Mpc^{-3} dex^{-1}
        Lstar = 10**(9.46) # Solar luminosity
        alpha = 1.00
        sigma = 0.50

        LIR = 10**log_LIR # solar luminosity

        Phi_IR = Phi_star * (LIR / Lstar)**(1 - alpha) * np.exp(-1 / (2 * sigma**2) * (np.log10(1 + LIR / Lstar))**2) # from Gruppioni paper eqn (3)  	

        return Phi_IR

    def PDF_log_Lgamma_given_log_LIR(log_LIR, log_Lgamma): #log_LIR = log_10(L_IR / solar_luminosity) # unitless
        LIR_solar_luminosity = 10**log_LIR # Solar luminosity
        L_IR_erg_second = LIR_solar_luminosity * 3.826e33 # erg/s

        a = 1.09
        g = 40.8
        sigma_SF = 0.202 

        mean = g + a * np.log10(L_IR_erg_second / 1e45)
        std = sigma_SF

        return norm.pdf(log_Lgamma, loc=mean, scale=std)

    def integrand(PhiIR_of_logLIRs, log_LIRs, log_Lgamma):
        return PhiIR_of_logLIRs * PDF_log_Lgamma_given_log_LIR(log_LIRs, log_Lgamma)

    PhiIR_of_logLIRs = Phi_IR(log_LIRs)

    for i in range(LFs.shape[0]):
        for j in range(LFs.shape[1]):
            LFs[i,j] = simpson(integrand(PhiIR_of_logLIRs, log_LIRs, np.log10(l_erg[i,j])), x=log_LIRs)
    return 1e-9 / np.log(10) / l * LFs # LF has spatial units of Mpc^{-3}. We need to convert this to kpc^{-3}. Hence the factor of 1e-9


def spec_SFG1(energy, params):
    Gamma = 2.2
    return energy**(-Gamma)

In [9]:
def ZL_mAGN(z, l, params):

    log_phi1 = params[1]
    phi1 = 10**log_phi1

    l_erg = l * E_photon_mAGN # erg/s
    LFs = np.zeros_like(l)

    def Phi_5G(log_L5G, z): #log_L5G = log_10(L_5GHz / (erg/s)) # unitless
        #Output is in Mpc^{-3}

        L_5G = 10**log_L5G # erg/s
        radio_bandwidth = 4.87e9 # measured in Hz # width of radio band centered around blueshifted frequency of 5GHz 
        diff_L5G = L_5G / radio_bandwidth * 1e-7 # measured in W/Hz # Converted erg to Joule # luminosity per unit frequency

        # Values taken from Table 4 of Yuan 2018 paper. Second row.
        p1 = 2.085
        p2 = -4.602
        z_c = 0.893
        k1 = 1.744
        e1 = ( (1+z_c)**p1 + (1+z_c)**p2 ) / ( ((1+z_c)/(1+z))**p1 + ((1+z_c)/(1+z))**p2 )
        e2 = (1+z)**k1
        # phi1 = 10**(-3.749) # Mpc^{-3}
        L_star = 10**21.592 # W/Hz
        beta = 0.139
        gamma = 0.878

        # From Yuan 2018 paper equation 21
        # Note that this is dN/dV dlog(diff_5G). But this is also equal to dN/dV dlog(L_5G) because the radio bandwidth is fixed.
        Phi_5G = e1 * phi1 * ( (diff_L5G / (e2 * L_star))**beta + (diff_L5G / (e2 * L_star))**gamma )**-1

        return Phi_5G
    

    def PDF_log_Lgamma_given_log_L5G(log_L5G, log_Lgamma): #log_L5G = log_10(L_5GHz / (erg/s)) # unitless
        L_5GHz = 10**log_L5G # erg/s

        b = 0.78
        d = 40.78
        sigma_mAGN = 0.880

        mean = d + b * np.log10(L_5GHz / 1e40)
        std = sigma_mAGN

        return norm.pdf(log_Lgamma, loc=mean, scale=std)
    

    def integrand(log_L5G, z, log_Lgamma):
        return Phi_5G(log_L5G, z) * PDF_log_Lgamma_given_log_L5G(log_L5G, log_Lgamma)
    


    for i in range(LFs.shape[0]):
        for j in range(LFs.shape[1]):
            LFs[i,j] = simpson(integrand(log_L5Gs, z[i,j], np.log10(l_erg[i,j])), x=log_L5Gs)


    return 1e-9 / np.log(10) / l * LFs # LF has spatial units of Mpc^{-3}. We need to convert this to kpc^{-3}. Hence the factor of 1e-9



def spec_mAGN(energy, params):
    Gamma = 2.2 #modified sepctrum to match the SFG spectrum
    return energy**(-Gamma)

In [10]:
als_SFG1 = [ZL_SFG1, spec_SFG1]
als_mAGN = [ZL_mAGN, spec_mAGN]
my_AEGIS.abun_lum_spec = [als_SFG1, als_mAGN]
my_AEGIS.source_class_list = ['extragalactic_isotropic_faint_single_spectrum', 'extragalactic_isotropic_faint_single_spectrum']

In [ ]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(params):

    input_params = params.numpy()

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-2)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 
    obs_info = {'psf_fits_path': '../../DGRB/FERMI_files/psf_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'edisp_fits_path': '../../DGRB/FERMI_files/edisp_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'event_type': 'PSF3', 'exposure_map': None}
    obs_photon_info = my_AEGIS.mock_observe(photon_info, obs_info)
    
    return obs_photon_info

In [12]:
def manual_simulate_for_sbi(proposal, num_simulations=1000, num_workers=32):
    """
    Simulates the model in parallel using joblib.
    Each simulation call samples a parameter from the proposal and passes the index to the simulator.
    """
    def run_simulation(i):
        if i % 10 == 0:
            print(f"i= {i}")
        # Sample a parameter from the proposal (sbi.utils.BoxUniform has a .sample() method)
        theta_i = proposal.sample()
        photon_info = simulator(theta_i)
        return theta_i, photon_info

    # Run simulations in parallel using joblib.
    results = Parallel(n_jobs=num_workers, timeout=None)(delayed(run_simulation)(i) for i in range(num_simulations))
    theta_list, photon_info_list = zip(*results)

    theta_tensor = torch.stack(theta_list, dim=0).to(torch.float32)
    
    
    return theta_tensor, photon_info_list

In [ ]:
# Define the prior using sbi.utils.BoxUniform
parameter_range = torch.tensor([[-5.0, -6.5],
                                [-1.0, -2.5]])

prior = utils.BoxUniform(low=parameter_range[0], high=parameter_range[1])

theta, train_photon_info_list = manual_simulate_for_sbi(prior,
                                   num_simulations=num_simulations,
                                   num_workers=num_workers)

i= 10
i= 0
i= 30
i= 20
i= 50
i= 70
i= 40
i= 60
i= 100
i= 90
i= 160
i= 120
i= 130
i= 80
i= 150
i= 170
i= 190
i= 140
i= 110
i= 180
i= 200
i= 210
i= 220
i= 230
i= 240
i= 250
i= 260
i= 270
i= 280
i= 290
i= 300
i= 310
i= 320
i= 330
i= 340
i= 350
i= 360
i= 370
i= 380
i= 390
i= 400
i= 410
i= 420
i= 430
i= 440
i= 450
i= 460
i= 470


/home/users/ids29/.conda/envs/sbi_env/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


i= 480
i= 490
i= 500
i= 510
i= 520
i= 530
i= 540
i= 550
i= 560
i= 570
i= 580
i= 590
i= 600
i= 610
i= 620
i= 630
i= 640
i= 650
i= 660
i= 670
i= 680
i= 690
i= 700
i= 710
i= 720
i= 730
i= 740
i= 750
i= 760
i= 770
i= 780
i= 790
i= 800
i= 810
i= 820
i= 830
i= 840
i= 850
i= 860
i= 870
i= 880
i= 890
i= 900
i= 910
i= 920
i= 930
i= 940
i= 950
i= 960
i= 970
i= 980
i= 990


In [ ]:
# 'photon_info_list' is a list of dictionaries

# Save to file
with open('/home/users/ids29/DGRB Scripts/training_photon_info_list_1SFG_1mAGN.pkl', 'wb') as f:
    pickle.dump(train_photon_info_list, f)

# Save to file
torch.save(theta, '/home/users/ids29/DGRB Scripts/thetas_1SFG_1mAGN.pt')
torch.save(parameter_range, '/home/users/ids29/DGRB Scripts/parameter_range_1SFG_1mAGN.pt')

In [15]:
test_params_1 = torch.tensor([-2.08, -3.749])
test_photon_info_1 = simulator(test_params_1)

In [16]:
test_params_2 = torch.tensor([-4.08, -5.55])
test_photon_info_2 = simulator(test_params_2)

In [17]:
torch.save(test_params_1, r'/home/users/ids29/DGRB Scripts/test_params_1SFG_1mAGN_part1.pt')

with open(r'/home/users/ids29/DGRB Scripts/test_photon_info_1SFG_1mAGN_part1.pkl', 'wb') as f:
    pickle.dump(test_photon_info_1, f)


torch.save(test_params_2, r'/home/users/ids29/DGRB Scripts/test_params_1SFG_1mAGN_part2.pt')

with open(r'/home/users/ids29/DGRB Scripts/test_photon_info_1SFG_1mAGN_part2.pkl', 'wb') as f:
    pickle.dump(test_photon_info_2, f)

In [ ]:
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import torch

# Define the six θ values
theta_values = [
    [-5.0, -6.5],
    [-5.0, -2.5],
    [-1.0, -6.5],
    [-1.0, -2.5],
    [-2.08, -3.749],
    [-4.08, -5.55]
]

# Convert to tensor
theta_tensors = [torch.tensor(theta) for theta in theta_values]

# Run simulations in parallel
results = Parallel(n_jobs=-1)(delayed(simulator)(theta) for theta in theta_tensors)

KeyError: 1

In [21]:
with open("simulator_results.pkl", "wb") as f:
    pickle.dump(results, f)